In [1]:
import pandas as pd
import re

In [2]:
raw_data = '../data/raw/'

In [3]:
sales_df = pd.read_pickle(raw_data + 'sales_df.pkl')
sales_df.head()

,PIN,FOLIO,DOR_CODE,NBHC,S_DATE,VI,QU,REA_CD,S_AMT,SUB,STR,S_TYPE,OR_BK,OR_PG,GRANTOR,GRANTEE,DOC_NUM
0,A-23-33-15-ZZZ-000000-00040.0,0000050000,8600,227002.0,1928-03-01,V,U,38,100.0,ZZZ,233315,QC,00810,0435,,HILLSBOROUGH COUNTY,
1,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2019-09-16,V,U,11,100.0,001,012717,DD,26961,0622,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY PAULINE L LIFE ESTATE,2019404632
2,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2014-04-01,V,U,11,100.0,001,012717,TR,22504,1677,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY STEPHEN R TRUSTEE,2014113139
3,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2006-12-05,V,U,11,100.0,001,012717,WD,17411,0866,SEVIGNY STEPHEN R,SEVIGNY STEPHEN R TRUSTEE,2007055159
4,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,1988-10-01,V,U,38,28000.0,001,012717,WD,05528,1424,,,88224230


In [4]:
print(sales_df.shape)
print(sales_df.info())

(2166282, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2166282 entries, 0 to 2166281
Data columns (total 17 columns):
 #   Column    Dtype  
---  ------    -----  
 0   PIN       object 
 1   FOLIO     object 
 2   DOR_CODE  object 
 3   NBHC      float64
 4   S_DATE    object 
 5   VI        object 
 6   QU        object 
 7   REA_CD    object 
 8   S_AMT     float64
 9   SUB       object 
 10  STR       object 
 11  S_TYPE    object 
 12  OR_BK     object 
 13  OR_PG     object 
 14  GRANTOR   object 
 15  GRANTEE   object 
 16  DOC_NUM   object 
dtypes: float64(2), object(15)
memory usage: 281.0+ MB
None


In [5]:
sales_df[sales_df['DOC_NUM'] == '2018356883'].T

,964451
PIN,U-10-29-20-29R-000001-00008.0
FOLIO,0662680766
DOR_CODE,0100
NBHC,220009.0
S_DATE,2018-08-28
VI,I
QU,Q
REA_CD,02
S_AMT,305000.0
SUB,29R


The PIN number is the parcel identifier and is formatted as follows.

Example PIN: A-16-28-19-3A3-000034-00004.0

A 16 28 19 3A3 000034 00004.0 (Municipality) (Section) (Township) (Range) (Land Type-ID) (Block# ) (Lot#)

The MUNICIPALITY CODE indicates whether the property is located in a city or unincorporated county. The code for the City of Tampa is "A", for the City of Temple Terrace "T" and for the City of Plant City "P". All unincorporated county is coded with "U".

The SECTION-TOWNSHIP-RANGE

The Land Type-ID is to further identify the type of land. All platted Subdivisions and Condominiums have been assigned an ID number. This number may be a combination of numbers and letters, such as "3A3". Not all property in the county is platted. For all unplatted, metes and bounds described lands a Land Type-ID of "ZZZ" has been assigned. This way, one look at the PIN number can tell you whether it is platted or unplatted land.

The BLOCK# is 6 digits and will hold the Block Number in platted Subdivisions or Building Number in a Condominium. For unplatted land having a Land Type-ID of "ZZZ" the BLOCK# will carry a uniquely assigned number.

The LOT# is 6 digits and will hold the Lot Numbers in Platted Subdivisions or Unit Numbers in a Condominium. For all unplatted land having a Land Type-ID of "ZZZ" the LOT# will carry a uniquely assigned number.

In [11]:
print(sales_df['PIN'].describe())

count                           2166282
unique                           504015
top       U-36-27-18-ZZZ-000000-75670.1
freq                                 91
Name: PIN, dtype: object


In [6]:
# Validate all PIN's match format (i.e. 'A-16-28-19-3A3-000034-00004.0')
pin = sales_df['PIN'].str.match("^[A-Z]-\d{2}-\d{2}-\d{2}-[A-Z0-9]{3}-[A-Z0-9]{6}-[A-Z0-9]{5}\.[A-Z0-9]$")
assert sales_df[~pin].shape[0] == 0

In [7]:
# Split PIN into the seven data elements that are used to create the PIN
sales_df[['MUNICIPALITY_CD', 'SECTION_CD', 'TOWNSHIP_CD', 'RANGE_CD', 'LAND_TYPE_ID', 'BLOCK_NUM', 'LOT_NUM']] = sales_df['PIN'].str.split('-', expand=True)
sales_df.head()

,PIN,FOLIO,DOR_CODE,NBHC,S_DATE,VI,QU,REA_CD,S_AMT,SUB,...,GRANTOR,GRANTEE,DOC_NUM,MUNICIPALITY_CD,SECTION_CD,TOWNSHIP_CD,RANGE_CD,LAND_TYPE_ID,BLOCK_NUM,LOT_NUM
0,A-23-33-15-ZZZ-000000-00040.0,0000050000,8600,227002.0,1928-03-01,V,U,38,100.0,ZZZ,...,,HILLSBOROUGH COUNTY,,A,23,33,15,ZZZ,000000,00040.0
1,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2019-09-16,V,U,11,100.0,001,...,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY PAULINE L LIFE ESTATE,2019404632,U,01,27,17,001,000000,00001.0
2,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2014-04-01,V,U,11,100.0,001,...,SEVIGNY STEPHEN R TRUSTEE,SEVIGNY STEPHEN R TRUSTEE,2014113139,U,01,27,17,001,000000,00001.0
3,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,2006-12-05,V,U,11,100.0,001,...,SEVIGNY STEPHEN R,SEVIGNY STEPHEN R TRUSTEE,2007055159,U,01,27,17,001,000000,00001.0
4,U-01-27-17-001-000000-00001.0,0000080000,0000,211007.0,1988-10-01,V,U,38,28000.0,001,...,,,88224230,U,01,27,17,001,000000,00001.0


In [15]:
print(sales_df['FOLIO'].describe())

count        2166282
unique        504015
top       0170630100
freq              91
Name: FOLIO, dtype: object


There are over 300 Department of Revenue (DOR) codes which represent the type of property.  The types of properties include single family homes, condos, malls, night clubs, vacant land, etc.  The DOR code is indicated in the DOR_C field in the parcel data.  

The residential property types that will be used for this analysis include:
- 0100: Single Family Residential
- 0102: Single family home built around a mobile home
- 0106: Townhouse/Villa
- 0200: Mobile Home
- 0400: Condominium
- 0408: Mobile Home Condominium
- 0800: Multi-Family Residential (Duplex, Triplex, Quadplex, etc.) < 10 units
- 0801: Multi-Family Residential (units individually owned)
- 0802: Multi-Family Residential (units rentals)

In [8]:
parcel_df = pd.read_pickle(raw_data + 'parcel_df.pkl')
parcel_df.head()

,FOLIO,TYPE,Edit_dt,PIN,DOR_C,OWNER,ADDR_1,ADDR_2,CITY,STATE,...,MUNI,SD1,SD2,TIF,BASE,S_DATE,VI,S_AMT,ACREAGE,NBHC
0,0000010000,,2019-04-08,A-23-33-15-ZZZ-000000-00020.0,8800,UNITED STATES,EGMONT KEY STATE PARK,4905 34TH ST S PMB 5000,SAINT PETERSBURG,FL,...,A,,,,0,None,,0.0,313.01100,227002.0
1,0000050000,,2019-04-08,A-23-33-15-ZZZ-000000-00040.0,8600,HILLSBOROUGH COUNTY,REAL ESTATE DEPT,PO BOX 1110,TAMPA,FL,...,A,,,,0,None,,0.0,6.14792,227002.0
2,0000080000,,2019-04-09,U-01-27-17-001-000000-00001.0,0000,PAULINE L SEVIGNY/ LIFE ESTATE,19931 ANGEL LN,,ODESSA,FL,...,U,,,,0,None,,0.0,4.71341,211007.0
3,0000080100,,2019-04-09,U-01-27-17-001-000000-00001.1,0100,JEFFERY AND PATRICIA SEVIGNY,19859 ANGEL LN,,ODESSA,FL,...,U,,,,2016,1987-08-01,I,50000.0,5.05878,211007.0
4,0000090000,,2019-04-09,U-01-27-17-001-000000-00002.0,0100,MARIETTA SHIVER LIFE ESTATE,19901 ANGEL LN,,ODESSA,FL,...,U,,,,1994,None,,0.0,1.00059,211007.0


In [9]:
print(parcel_df.shape)
print(parcel_df.info())

(512207, 47)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512207 entries, 0 to 512206
Data columns (total 47 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   FOLIO      512207 non-null  object 
 1   TYPE       512207 non-null  object 
 2   Edit_dt    512207 non-null  object 
 3   PIN        512207 non-null  object 
 4   DOR_C      512207 non-null  object 
 5   OWNER      512207 non-null  object 
 6   ADDR_1     512207 non-null  object 
 7   ADDR_2     512207 non-null  object 
 8   CITY       512207 non-null  object 
 9   STATE      512207 non-null  object 
 10  ZIP        512207 non-null  object 
 11  COUNTRY    512207 non-null  object 
 12  SUB        512207 non-null  object 
 13  SITE_ADDR  512207 non-null  object 
 14  SITE_CITY  512207 non-null  object 
 15  SITE_ZIP   512207 non-null  object 
 16  LEGAL1     512207 non-null  object 
 17  LEGAL2     512207 non-null  object 
 18  LEGAL3     512207 non-null  object 
 19  LEGAL4    

In [10]:
parcel_df[parcel_df['FOLIO'] == '0662680766'].T

,228527
FOLIO,0662680766
TYPE,
Edit_dt,2019-03-20
PIN,U-10-29-20-29R-000001-00008.0
DOR_C,0100
OWNER,MICHAEL P AND KERRY R WARD
ADDR_1,861 BAYOU VIEW DR
ADDR_2,
CITY,BRANDON
STATE,FL


In [14]:
print(parcel_df['FOLIO'].describe())

count         512207
unique        512207
top       0000010000
freq               1
Name: FOLIO, dtype: object


In [13]:
print(parcel_df['PIN'].describe())

count     512207
unique    512180
top             
freq          28
Name: PIN, dtype: object
